In [1]:
from database_utils import add_spec_from_path_to_database
from database_functions import create_continuous_aggregate_sql, refresh_continuous_aggregate, get_table_names_sql, drop_materialized_view_sql
from tqdm import tqdm

2023-09-11 10:10:07,162 [INFO] logging_utils.py/<module>:56 >> Starting ipykernel_launcher.


In [ ]:
drop_materialized_view_sql('alaska_anchorage_01_1minute_max')

In [2]:
for table in tqdm(get_table_names_sql()):
    view_name = table + '_daily_row_count'
    assert '_daily_row_count' in view_name
    drop_materialized_view_sql(view_name)

In [ ]:
for table in tqdm(get_table_names_sql()):
    create_continuous_aggregate_sql(table, timebucket='1minute', agg_function='MAX')

In [2]:
create_continuous_aggregate_sql(table='alaska_anchorage_01', timebucket='1minute', agg_function='MAX')


    CREATE MATERIALIZED VIEW alaska_anchorage_01_1minute_MAX WITH (timescaledb.continuous) AS
    SELECT time_bucket(INTERVAL '1minute', datetime) AS time,
           MAX("5.563000202178955") AS "5.563000202178955",MAX("5.875") AS "5.875",MAX("6.188000202178955") AS "6.188000202178955",MAX("6.5") AS "6.5",MAX("6.813000202178955") AS "6.813000202178955",MAX("7.125") AS "7.125",MAX("7.438000202178955") AS "7.438000202178955",MAX("7.75") AS "7.75",MAX("8.062999725341797") AS "8.062999725341797",MAX("8.375") AS "8.375",MAX("8.687999725341797") AS "8.687999725341797",MAX("9.0") AS "9.0",MAX("9.312999725341797") AS "9.312999725341797",MAX("9.625") AS "9.625",MAX("9.937999725341797") AS "9.937999725341797",MAX("10.25") AS "10.25",MAX("10.562999725341797") AS "10.562999725341797",MAX("10.875") AS "10.875",MAX("11.187999725341797") AS "11.187999725341797",MAX("11.5") AS "11.5",MAX("11.812999725341797") AS "11.812999725341797",MAX("12.125") AS "12.125",MAX("12.437999725341797") AS "12.437999725